In [1]:
import pandas as pd
import gzip
from functions.columns import *

C:\Users\Majd0397\Documents\00 - GIT SERVICIO SOCIAL\00 - Dash\BSRN_IGEF\Interfaz (Dash)\06 Sexta_Version\functions\columns.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Conexión exitosa


AttributeError: 'Series' object has no attribute 'day'

In [6]:
def new_df(dff,columns_1,columns_2):
    """
    Crea un nuevo DataFrame combinando columnas de `dff` según reglas específicas.

    Parámetros:
    - dff: DataFrame. El DataFrame original.
    - columns_1: Lista. Lista de nombres de columnas para el primer subconjunto de columnas.
    - columns_2: Lista. Lista de nombres de columnas para el segundo subconjunto de columnas.

    Retorna:
    - DataFrame. Un nuevo DataFrame combinado según las reglas especificadas.

    Descripción:
    Esta función toma un DataFrame `dff` y dos listas de nombres de columnas `columns_1` y `columns_2`.
    A partir de estas listas, se crean dos DataFrames `dff1` y `dff2` que contienen las columnas
    correspondientes de `dff`. Luego, se asignan nuevos índices a cada DataFrame y se renombran las columnas
    con números enteros en ambos DataFrames.

    Posteriormente, se realiza una concatenación vertical de los DataFrames `dff1` y `dff2` en el orden
    especificado por los índices. Finalmente, se ordena el DataFrame resultante según los índices y se retorna.
    """
    
    dff1=dff[columns_1]
    dff2=dff[columns_2]

    index1=[x for x in np.arange(0,len(dff1)*2,2)]
    index2=[x for x in np.arange(1,len(dff2)*2,2)]

    dff1 = dff1.set_index(pd.Index(index1))
    dff2 = dff2.set_index(pd.Index(index2))

    dff1.columns=[x for x in range(0,len(dff1.columns))]
    dff2.columns=[x for x in range(0,len(dff2.columns))]
    dff3 = pd.concat([dff1, dff2], ignore_index=False, axis=0)
    dff3 = dff3.sort_index()
    return dff3


#Funcion para comprimir archivo
def CompressFile(name_file, name_file_compress):
    print("Comprimiendo archivo")
    with open(name_file, 'rb') as file:
        with gzip.open(name_file_compress, 'wb') as file_compress:
            file_compress.writelines(file)
    
    print("Archivo comprimido exitosamente: ", name_file_compress)

In [2]:
rounds=dict(round_int=["GLOBAL_Avg","GLOBAL_Min","GLOBAL_Max","DIRECT_Avg","DIRECT_Min","DIRECT_Max","DIFFUSE_Avg",
                       "DIFFUSE_Min","DIFFUSE_Max","DOWNWARD_Avg","DOWNWARD_Min","DOWNWARD_Max","UPWARD_SW_Avg",
                       "UPWARD_SW_Min","UPWARD_SW_Max","UPWARD_LW_Avg","UPWARD_LW_Min","UPWARD_LW_Max","PRESSURE_Avg",
                       "UPWARD_SW_Avg","UPWARD_SW_Min","UPWARD_SW_Max","UPWARD_LW_Avg","UPWARD_LW_Min","UPWARD_LW_Max",],
           round_float1=["GLOBAL_Std","DIRECT_Std","DIFFUSE_Std","DOWNWARD_Std","AIR_TEMPERATURE_Avg",
                         "RELATIVE_HUMIDITY_Avg","UPWARD_SW_Std","UPWARD_LW_Std","UVB_Avg","UVB_Std","UVB_Min","UVB_Max",
                        "DWTERMO_Avg","UWTERMO_Avg"],
           round_float2=["DWIRTEMPC_Avg","UWIRTEMPC_Avg"])


#########################################
#  Opciones para categorias de archivo  #
#########################################

options=dict(C0100=["day","minute","GLOBAL_Avg","GLOBAL_Std","GLOBAL_Min","GLOBAL_Max","DIRECT_Avg","DIRECT_Std",
                    "DIRECT_Min","DIRECT_Max","DIFFUSE_Avg","DIFFUSE_Std","DIFFUSE_Min","DIFFUSE_Max","DOWNWARD_Avg",
                    "DOWNWARD_Std","DOWNWARD_Min","DOWNWARD_Max","AIR_TEMPERATURE_Avg","RELATIVE_HUMIDITY_Avg",
                    "PRESSURE_Avg","empy"],
             C0200=["day","minute","GLOBAL_Avg","GLOBAL_Std","GLOBAL_Min","GLOBAL_Max"],
             C0300=["day","minute","UPWARD_SW_Avg","UPWARD_SW_Std","UPWARD_SW_Min","UPWARD_SW_Max","UPWARD_LW_Avg",
                    "UPWARD_LW_Std","UPWARD_LW_Min","UPWARD_LW_Max","null_int","null_float1","null_int", "null_int"],
             C0500=["day","minute","null_float1", "UVB_Avg","UVB_Std","UVB_Min","UVB_Max","empy"], 
             C4000=["day","minute","DWIRTEMPC_Avg","null_float2","null_float2","null_float2","DWTERMO_Avg",
                    "UWIRTEMPC_Avg","null_float2","null_float2","null_float2","UWTERMO_Avg","empy"])

#########################################
#         Opciones para espacio         #
#########################################
spaces=dict(C0100=[3,4,6,5,4,4,6,5,4,4,8,5,4],
            C0300=[3,4,6,5,4,4,6,5,4,4,6,5,4,4],
            C0500=[3,4,5,5,5,5,5,5,5,5,5,5,5,5],
            C4000=[3,4,6,5,5,5,5,7,5,5,5,6])



In [7]:
#Se agregan columnas de minutos y dias

day = [x.day for x in df["TIMESTAMP"]]
minute = [x.hour*60+x.minute for x in df["TIMESTAMP"]]
month = [x.month for x in df["TIMESTAMP"]]

df_aux=df.copy()
df_aux.insert(0,"minute",minute)
df_aux.insert(0,"day",day)

In [8]:
#Se redondean segun opciones y se reemplazan los datos null
for i in rounds["round_int"]:
    df_aux[i]=df_aux[i].fillna(-999)
    df_aux[i]=df_aux[i].round(0)
    df_aux[i]=df_aux[i].astype(int)

for i in rounds["round_float1"]:
    df_aux[i]=df_aux[i]+0.01
    df_aux[i]=df_aux[i].fillna(-99.9)
    df_aux[i]=df_aux[i].round(1)

for i in rounds["round_float2"]:
    df_aux[i]=df_aux[i].fillna(-99.99)
    df_aux[i]=df_aux[i].round(2)

In [5]:
dff=df_aux[list(options["C0300"])]

In [197]:
option="C0300"
dff=dff.astype(str)
for k in dff.columns:
    if k in rounds["round_float2"]:
        dff[k]=dff[k].astype(float).map("{:.2f}".format)

dff.columns=list(range(0,len(dff.columns)))
for j in dff.columns:
    dff[j]=dff[j].apply(lambda x: correct_spaces(x, spaces[option][j]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
10440,1,0,4,0.2,4,5,469,0.3,469,470,-999,-99.9,-999,-999
10441,1,1,4,0.2,3,4,470,0.3,469,470,-999,-99.9,-999,-999
10442,1,2,3,0.1,3,4,469,0.2,469,470,-999,-99.9,-999,-999
10443,1,3,3,0.0,3,3,469,0.3,468,469,-999,-99.9,-999,-999
10444,1,4,3,0.0,3,3,469,0.1,468,469,-999,-99.9,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55075,31,1435,3,0.0,3,3,477,0.1,476,477,-999,-99.9,-999,-999
55076,31,1436,3,0.0,3,3,476,0.4,475,477,-999,-99.9,-999,-999
55077,31,1437,3,0.0,3,3,476,0.2,475,476,-999,-99.9,-999,-999
55078,31,1438,3,0.1,3,3,476,0.3,476,477,-999,-99.9,-999,-999


In [203]:
dff[14]=dff[3].astype(str)

In [207]:
dff[15]=dff[14].apply(lambda x:len(x))

In [224]:
dff[15].index[dff[15]==max(dff[15])]

Int64Index([24779, 24780, 24781, 24782, 24783, 24784, 24785, 24786, 24787,
            24788, 24789, 24790],
           dtype='int64')

In [241]:
dff.loc[24790]

0         10
1       1390
2       -999
3     -999.0
4       -999
5       -999
6        467
7        1.0
8        466
9        469
10      -999
11     -99.9
12      -999
13      -999
14    -999.0
15         6
Name: 24790, dtype: object

In [11]:
list(df.columns)

['TIMESTAMP',
 'RECORD',
 'ZenDeg',
 'AzDeg',
 'AirMass',
 'CRBattVolt_Avg',
 'CRPTemp_Avg',
 'ProcessTime_s_Max',
 'GLOBAL_Avg',
 'GLOBAL_Std',
 'GLOBAL_Min',
 'GLOBAL_Max',
 'DIRECT_Avg',
 'DIRECT_Std',
 'DIRECT_Min',
 'DIRECT_Max',
 'DIFFUSE_Avg',
 'DIFFUSE_Std',
 'DIFFUSE_Min',
 'DIFFUSE_Max',
 'UPWARD_SW_Avg',
 'UPWARD_SW_Std',
 'UPWARD_SW_Min',
 'UPWARD_SW_Max',
 'DOWNWARD_Avg',
 'DOWNWARD_Std',
 'DOWNWARD_Min',
 'DOWNWARD_Max',
 'DWIRTEMP_Avg',
 'DWIRTEMP_Std',
 'DWIRTEMP_Min',
 'DWIRTEMP_Max',
 'UPWARD_LW_Avg',
 'UPWARD_LW_Std',
 'UPWARD_LW_Min',
 'UPWARD_LW_Max',
 'UWIRTEMP_Avg',
 'UWIRTEMP_Std',
 'UWIRTEMP_Min',
 'UWIRTEMP_Max',
 'DNITEMPC_Avg',
 'DNITEMPC_Std',
 'DNITEMPC_Min',
 'DNITEMPC_Max',
 'UVB_Avg',
 'UVB_Std',
 'UVB_Min',
 'UVB_Max',
 'SOIL_Avg',
 'AIR_TEMPERATURE_Avg',
 'RELATIVE_HUMIDITY_Avg',
 'DEW_POINT_Avg',
 'PRESSURE_Avg',
 'GLOBTT_Avg',
 'GLOBTT_Std',
 'GLOBTT_Min',
 'GLOBTT_Max',
 'GH_IRCOR_Avg',
 'GH_IRCOR_Std',
 'GH_IRCOR_Min',
 'GH_IRCOR_Max',
 'DH_IRCOR_

In [251]:
day

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [252]:
for i in df.columns:
    print(i)

TIMESTAMP
RECORD
ZenDeg
AzDeg
AirMass
CRBattVolt_Avg
CRPTemp_Avg
ProcessTime_s_Max
GLOBAL_Avg
GLOBAL_Std
GLOBAL_Min
GLOBAL_Max
DIRECT_Avg
DIRECT_Std
DIRECT_Min
DIRECT_Max
DIFFUSE_Avg
DIFFUSE_Std
DIFFUSE_Min
DIFFUSE_Max
UPWARD_SW_Avg
UPWARD_SW_Std
UPWARD_SW_Min
UPWARD_SW_Max
DOWNWARD_Avg
DOWNWARD_Std
DOWNWARD_Min
DOWNWARD_Max
DWIRTEMP_Avg
DWIRTEMP_Std
DWIRTEMP_Min
DWIRTEMP_Max
UPWARD_LW_Avg
UPWARD_LW_Std
UPWARD_LW_Min
UPWARD_LW_Max
UWIRTEMP_Avg
UWIRTEMP_Std
UWIRTEMP_Min
UWIRTEMP_Max
DNITEMPC_Avg
DNITEMPC_Std
DNITEMPC_Min
DNITEMPC_Max
UVB_Avg
UVB_Std
UVB_Min
UVB_Max
SOIL_Avg
AIR_TEMPERATURE_Avg
RELATIVE_HUMIDITY_Avg
DEW_POINT_Avg
PRESSURE_Avg
GLOBTT_Avg
GLOBTT_Std
GLOBTT_Min
GLOBTT_Max
GH_IRCOR_Avg
GH_IRCOR_Std
GH_IRCOR_Min
GH_IRCOR_Max
DH_IRCOR_Avg
DH_IRCOR_Std
DH_IRCOR_Min
DH_IRCOR_Max
RN_Avg
RN_Std
RN_Min
RN_Max
WS_ms_Mean
Wdir_Mean
Wdir_Std
WS_Std
WS_Max
WD_SMM
PPT_Tot
PPT_Accum
GH_CALC_Avg
UVTEMP_Avg
UVSIGNAL_Avg
DWSIGNAL_Avg
UWSIGNAL_Avg
DWIRTEMPC_Avg
DWTERMO_Avg
UWIRTEMPC_Avg
UWTE

In [12]:
mes=12

In [14]:
len(str(mes))

2

In [38]:
def aux(x,_):
    if x > 0:
        return x+0.01
    elif x<0:
        return x-0.01
    

df = pd.DataFrame([(.25, -.35), (.05, .67), (.66, .05), (.21, .18)],
                  columns=['dogs', 'cats'])
df["cats"].apply(lambda x: aux(x,1))

0   -0.36
1    0.68
2    0.06
3    0.19
Name: cats, dtype: float64

In [26]:
df["dogs"].round(1)

0    0.2
1    0.0
2    0.7
3    0.2
Name: dogs, dtype: float64

In [49]:
dff["UPWARD_SW_Avg"].iloc[0:10]

10440    0
10441    0
10442    0
10443    0
10444    0
10445    0
10446    0
10447    0
10448    0
10449    0
Name: UPWARD_SW_Avg, dtype: int32

In [3]:
df

,TIMESTAMP,RECORD,ZenDeg,AzDeg,AirMass,CRBattVolt_Avg,CRPTemp_Avg,ProcessTime_s_Max,GLOBAL_Avg,GLOBAL_Std,...,UVBw_Min,UVBw_Max,dif_GH_CALC_GLOBAL,ratio_GH_CALC_GLOBAL,sum_SW,porcent_2,empy,null_int,null_float1,null_float2
0,2022-01-01 00:00:00,419475,93.5127,247.085,21.350,12.38,310.28,1.130,-1.98739,0.050,...,None,None,-0.32485,1.163456,-2.296180,-0.045924,,-999,-99.9,-99.99
1,2022-01-01 00:01:00,419476,93.7393,247.147,16.520,12.38,310.26,1.129,-2.16432,0.050,...,None,None,-0.26898,1.124279,-2.416302,-0.048326,,-999,-99.9,-99.99
2,2022-01-01 00:02:00,419477,93.9654,247.209,12.600,12.38,310.25,1.117,-2.32640,0.056,...,None,None,-0.30379,1.130584,-2.610518,-0.052210,,-999,-99.9,-99.99
3,2022-01-01 00:03:00,419478,94.1904,247.270,9.480,12.38,310.23,1.117,-2.47822,0.031,...,None,None,-0.25973,1.104805,-2.718688,-0.054374,,-999,-99.9,-99.99
4,2022-01-01 00:04:00,419479,94.4159,247.331,7.004,12.38,310.22,1.128,-2.57639,0.036,...,None,None,-0.38640,1.149977,-2.941690,-0.058834,,-999,-99.9,-99.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56895,2023-01-09 12:15:00,310554,95.8135,111.339,0.204,12.32,295.23,1.106,-1.70784,0.012,...,None,None,-0.84140,1.492669,-2.531589,-0.050632,,-999,-99.9,-99.99
56896,2023-01-09 12:16:00,310555,95.5874,111.398,0.580,12.32,295.22,1.106,-1.68729,0.023,...,None,None,-0.83534,1.495078,-2.505228,-0.050105,,-999,-99.9,-99.99
56897,2023-01-09 12:17:00,310556,95.3613,111.457,1.134,12.32,295.21,1.115,-1.63477,0.020,...,None,None,-0.85459,1.522759,-2.473495,-0.049470,,-999,-99.9,-99.99
56898,2023-01-09 12:18:00,310557,95.1350,111.517,1.889,12.32,295.20,1.114,-1.59481,0.017,...,None,None,-0.87459,1.548398,-2.453403,-0.049068,,-999,-99.9,-99.99


In [70]:
dff["day"].iloc[10000]

7

In [73]:
dff["day"].

10480

In [78]:
dff["day"][dff["UPWARD_SW_Avg"]==-1]

10466     1
10468     1
10469     1
10471     1
10472     1
         ..
55075    31
55076    31
55077    31
55078    31
55079    31
Name: day, Length: 2724, dtype: int64